In [1]:
import os
from bs4 import BeautifulSoup

In [2]:
slozka = "filmy"

In [3]:
def seznam_souboru(slozka):
    seznam = []
    for file in os.listdir(slozka):
        seznam.append(file)
    return(seznam)

In [4]:
seznam_filmu = seznam_souboru(slozka)
seznam_filmu[0:10]

['10144-tabor-padlych-zen.html',
 '10173-eastern-krvavy-hugo.html',
 '102282-o-jecminkovi.html',
 '102314-zimni-vila.html',
 '10242-narkoblues.html',
 '103341-jan-hus-mse-za-tri-mrtve-muze.html',
 '103354-obcansky-prukaz.html',
 '103358-7-dni-hrichu.html',
 '103360-vojta-lavicka-nahoru-a-dolu.html',
 '103369-kuky-se-vraci.html']

## Začínáme scrapovat

### Nejdřív zjistíme, co všechno budeme scrapovat

In [5]:
kolik = 10

(Čím menší číslo, tím menší vzorek a tím rychlejší test. Pro zpracování kompletního seznamu nutno změnit proměnou na None.)

In [6]:
def jake_jsou_sekce(slozka):
    sekce = []
    for s in seznam_souboru(slozka)[:kolik]:
        with open(os.path.join(slozka, s), "r", encoding="utf-8") as f:
            soup = BeautifulSoup(f, "html.parser")
            item_details = soup.find_all(class_='db-film-section')
            for item_detail in item_details:
                h2_tags = item_detail.find_all('h2')
                for h2_tag in h2_tags:
                    if h2_tag.get_text() not in sekce:
                        sekce.append(h2_tag.get_text())
    return(sekce)

In [7]:
sekce = jake_jsou_sekce("filmy")

In [8]:
def jake_jsou_detaily(slozka):
    detaily = []
    for s in seznam_souboru(slozka)[:kolik]:
        with open(os.path.join(slozka, s), "r", encoding="utf-8") as f:
            soup = BeautifulSoup(f, "html.parser")
            item_details = soup.find_all(class_='item-details')
            for item_detail in item_details:
                h3_tags = item_detail.find_all('h3')
                for h3_tag in h3_tags:
                    if h3_tag.get_text() not in detaily:
                        detaily.append(h3_tag.get_text())
    return(detaily)

In [9]:
detaily = jake_jsou_detaily(slozka)

In [10]:
def jake_jsou_kredity(slozka):
    kredity = []
    for s in seznam_souboru(slozka)[:kolik]:
        with open(os.path.join(slozka, s), "r", encoding="utf-8") as f:
            soup = BeautifulSoup(f, "html.parser")
            item_details = soup.find_all(class_='db-film-credits-section')
            for item_detail in item_details:
                if "Štáb a tvůrci" in item_detail.text:
                    h3_tags = item_detail.find_all('h3')
                    for h3_tag in h3_tags:
                        if h3_tag.get_text() not in kredity:
                            kredity.append(h3_tag.get_text())
    return(kredity)

In [11]:
kredity = jake_jsou_kredity(slozka)

In [12]:
def jake_jsou_technicke_detaily(slozka):
    technika = []
    for s in seznam_souboru(slozka)[:kolik]:
        with open(os.path.join(slozka, s), "r", encoding="utf-8") as f:
            soup = BeautifulSoup(f, "html.parser")
            item_details = soup.find_all(class_='db-film-credits-section')
            for item_detail in item_details:
                if "Technické údaje" in item_detail.text:
                    h3_tags = item_detail.find_all('h3')
                    for h3_tag in h3_tags:
                        if h3_tag.get_text() not in technika:
                            technika.append(h3_tag.get_text())
    return(technika)

In [13]:
technika = jake_jsou_technicke_detaily(slozka)

In [14]:
technika

['Délka',
 'Minutáž',
 'Distribuční nosič',
 'Poměr stran',
 'Barva',
 'Zvuk',
 'Zvukový systém/formát',
 'Verze',
 'Mluveno',
 'Podtitulky',
 'Úvodní/závěrečné titulky',
 'Původní metráž',
 'Mezititulky',
 'Animační technika']

## Scrapování filmů

In [15]:
def id_persony(url):
    if "cs/person/" in url:
        id_persony = url.split("/")[-2]
    else:
        id_persony = 0
    return(id_persony)

In [16]:
def pridej_atributy(slovnik, seznam, nalezene):
    for s in seznam:
        for n in nalezene:
            if s in n:
                n = n.replace(s," ")
                slovnik[s] = n.strip()

In [17]:
def scrape_film(slozka, soubor):

    with open(os.path.join(slozka, soubor), "r", encoding="utf-8") as f:
        soup = BeautifulSoup(f, "html.parser")
    
    atributy = {}
    
    atributy["Film"] = f"""{soup.find("h1").text.strip()} ({soup.find('meta', {'property': 'og:url'})['content'].split("/")[-2]})"""

    ## Detaily – země původu, copyright…
    
    item_details = soup.find_all(class_='item-details')
    divs_text = []

    for item_detail in item_details:
        divs = item_detail.find_all('div')
        for div in divs:
            obsah = div.text
            if div.find("span"):
                obsah = div.find("h3").text + ", ".join([s.text for s in div.find_all(class_="comma")])
            divs_text.append(obsah)

    pridej_atributy(atributy, detaily, divs_text)
    
    ## Credits
    
    item_details = soup.find_all(class_='db-film-credits-section')
    divs_text = []

    for item_detail in item_details:
        
        if "Štáb a tvůrci" in item_detail.find("h2").text:
            divs = item_detail.find_all('div')
            for div in divs:
                if div.find_all("a"):
                    prvni = div.find("h3").text
                    odkazy = div.find_all("a")
                    odkazy2 = []
                    for a in odkazy:
                        id = id_persony(a["href"])
                        odkazy2.append(f"{a.text} ({id})")
                        druhy = ", ".join(odkazy2)
                        divs_text.append(prvni + " " + druhy)
                else:
                    divs_text.append(div.text.strip())
            
            pridej_atributy(atributy, kredity, divs_text)
            
        if "Technické údaje" in item_detail.find("h2").text or "Produkční údaje" in item_detail.find("h2").text:
            divs = item_detail.find_all('div')
            for div in divs:
                if div.find("h3"):
                    if div.find("p"):
                        atributy[div.find("h3").text.strip()] = div.find("p").text.strip()
                        if "Žánr" in div.find("h3").text:
                            zanry = [z.strip() for z in div.find("p").text.split(",")]
                            zanry = ", ".join(zanry)
                            atributy[div.find("h3").text.strip()] = zanry
                        if div.find(class_="comma"):
                            atributy[div.find("h3").text.strip()] = ", ".join([s.text.strip() for s in div.find_all(class_="comma")])
                        
        if "Písně" in item_detail.find("h2").text:
            pisne = item_detail.find_all('h3')
            pisne = [pisen.text.strip() for pisen in pisne]
            atributy["Písně"] = pisne
            
        if "Lokace" in item_detail.find("h2").text:
            lokace = item_detail.find('p')
            atributy["Lokace"] = lokace.text.strip()

        if "Ocenění" in item_detail.find("h2").text:
            oceneni = item_detail.find_all(class_="col-md-3")
            oceneni = [cena.text.strip() for cena in oceneni]
            atributy["Ocenění"] = oceneni
            
        if "Hudba" in item_detail.find("h2").text:
            divs = item_detail.find_all('div')
            for div in divs:
                atributy[div.find("h3").text] = [f"""{a.text} ({id_persony(a["href"])})""" for a in div.find_all("a")]
            
    item_details = soup.find_all(class_='film-description')
    
    for item_detail in item_details:
    
        if "Anotace" in item_detail.find("h2").text:
            anotace = item_detail.find('p')            
            atributy["Anotace"] = anotace.text.strip()
            
        if "Obsah" in item_detail.find("h2").text:
            obsah = item_detail.find('p')            
            atributy["Obsah"] = obsah.text.strip()
            
        
    return(atributy)

## Test

In [18]:
test = scrape_film(slozka, "9013-kolja.html")
test

{'Film': 'Kolja (9013)',
 'Země původu': 'Česká republika,                                                             Velká Británie,                                                             Francie',
 'Copyright': '1996',
 'Rok výroby': '1995',
 'Minutáž': '108 min',
 'Režie': 'Jan Svěrák (476)',
 'Kategorie': 'film',
 'Žánr': 'komedie, smutná komedie',
 'Typologie': 'hraný, distribuční',
 'Originální název': 'Kolja',
 'Český název': 'Kolja',
 'Anglický název': 'Kolya',
 'Pomocná režie': 'David Rauch (1810)',
 'Asistent režie': 'Slávek Horák (80123), Nikola Hejko (7510)',
 'Původní filmový námět': 'Pavel Taussig (11546)',
 'Scénář': 'Zdeněk Svěrák (1892)',
 'Dramaturg': 'Václav Šašek (1766), Anna Vášová (37301)',
 'Kamera': 'Vladimír Smutný (1772)',
 'Asistent kamery': 'Jan Carda (7004)',
 'Architekt': 'Miloš J. Kohout (5609)',
 'Návrhy kostýmů': 'Katarína Hollá (6534)',
 'Masky': 'František Příhoda (7109)',
 'Střih': 'Alois Fišárek (570)',
 'Zvuk': 'zvukový',
 'Producent': 'Jan S

## Todo

- víc odstavců anotace a obsahu
- víc režisérů a žánrů do seznamu?

## Konečně DataFrame

In [19]:
import pandas as pd

In [20]:
kolik = None

In [21]:
def frejm(slozka):
    df = pd.DataFrame()
    total = len(seznam_souboru(slozka)[:kolik])
    count = 0
    for s in seznam_souboru(slozka)[:kolik]:
        count = count + 1
        print(f"{count} / {total} / {s}")
        line = pd.DataFrame.from_dict([scrape_film(slozka, s)])
        df = pd.concat([df, line])
    df = df.reset_index(drop = True)
    return(df)

In [22]:
df = frejm("filmy")
df

1 / 4536 / 10144-tabor-padlych-zen.html
2 / 4536 / 10173-eastern-krvavy-hugo.html
3 / 4536 / 102282-o-jecminkovi.html
4 / 4536 / 102314-zimni-vila.html
5 / 4536 / 10242-narkoblues.html
6 / 4536 / 103341-jan-hus-mse-za-tri-mrtve-muze.html
7 / 4536 / 103354-obcansky-prukaz.html
8 / 4536 / 103358-7-dni-hrichu.html
9 / 4536 / 103360-vojta-lavicka-nahoru-a-dolu.html
10 / 4536 / 103369-kuky-se-vraci.html
11 / 4536 / 103372-habermannuv-mlyn.html
12 / 4536 / 103374-dobre-placena-prochazka.html
13 / 4536 / 103402-nebe-peklo.html
14 / 4536 / 103495-lidice.html
15 / 4536 / 103499-medvedi-ostrovy.html
16 / 4536 / 103509-dont-stop.html
17 / 4536 / 104213-az-na-veky.html
18 / 4536 / 10574-elfilm.html
19 / 4536 / 106663-dum.html
20 / 4536 / 106666-soukromy-vesmir.html
21 / 4536 / 106667-mamas-papas.html
22 / 4536 / 10781-karu-suda-medvedi-srdce.html
23 / 4536 / 10960-unos-domu.html
24 / 4536 / 1118-nachove-plachty.html
25 / 4536 / 11221-mechanika.html
26 / 4536 / 112328-jmenem-krale.html
27 / 4536 / 

205 / 4536 / 180001-vsechno-je-sracka.html
206 / 4536 / 180025-laska-v-hrobe.html
207 / 4536 / 180317-rekvalifikace.html
208 / 4536 / 180386-muzi-v-nadeji.html
209 / 4536 / 18042-ctyri-lasky.html
210 / 4536 / 18045-fimfarum-jana-wericha.html
211 / 4536 / 180492-bastardi-2.html
212 / 4536 / 180545-svata-ctverice.html
213 / 4536 / 18056-franta-nebojsa.html
214 / 4536 / 18058-splneny-sen.html
215 / 4536 / 180585-okresni-prebor-posledni-zapas-pepika-hnatka.html
216 / 4536 / 180618-probudim-se-vcera.html
217 / 4536 / 181233-ve-stinu-bohyne-matky-zeme.html
218 / 4536 / 181912-zneuzivany.html
219 / 4536 / 181928-skolni-vylet.html
220 / 4536 / 181956-co-chytnes-v-zite.html
221 / 4536 / 181960-osina.html
222 / 4536 / 181961-cesta.html
223 / 4536 / 181962-stoh.html
224 / 4536 / 18203-z-mesta-cesta.html
225 / 4536 / 182231-libas-jako-dabel.html
226 / 4536 / 18262-pupendo.html
227 / 4536 / 18264-jedna-ruka-netleska.html
228 / 4536 / 18265-utek-do-budina.html
229 / 4536 / 182826-laska-je-laska.html

412 / 4536 / 395109-dostavenicko-ve-mlynici.html
413 / 4536 / 395110-smich-a-plac.html
414 / 4536 / 395111-vystavni-parkar-a-lepic-plakatu.html
415 / 4536 / 395112-to-nejlepsi-cislo.html
416 / 4536 / 395113-jarni-sen-stareho-mladence.html
417 / 4536 / 395114-hubicka.html
418 / 4536 / 395115-ponrepovo-kouzelnictvi.html
419 / 4536 / 395116-rudi-na-krtinach.html
420 / 4536 / 395117-rudi-na-zaletech.html
421 / 4536 / 395118-rudi-se-zeni.html
422 / 4536 / 395119-rudi-sportsman.html
423 / 4536 / 395120-sokove.html
424 / 4536 / 395121-ctyri-rocni-doby.html
425 / 4536 / 395122-dama-s-barzojem.html
426 / 4536 / 395123-falesny-hrac.html
427 / 4536 / 395124-pro-penize.html
428 / 4536 / 395125-zahadny-zlocin.html
429 / 4536 / 395126-zive-modely.html
430 / 4536 / 395127-americky-souboj.html
431 / 4536 / 395128-cholera-v-praze.html
432 / 4536 / 395129-estrella.html
433 / 4536 / 395130-faust.html
434 / 4536 / 395131-idyla-ze-stare-prahy.html
435 / 4536 / 395132-konec-milovani.html
436 / 4536 / 395133

616 / 4536 / 395314-zkouska.html
617 / 4536 / 395315-bud-pripraven.html
618 / 4536 / 395316-carovne-oci.html
619 / 4536 / 395317-hrabenka-filangieri.html
620 / 4536 / 395318-kdyz-srdce-promluvi.html
621 / 4536 / 395319-madame-golvery.html
622 / 4536 / 395320-muz-bez-srdce.html
623 / 4536 / 395321-palicova-dcera.html
624 / 4536 / 395322-pepanek-nezdara.html
625 / 4536 / 395323-pro-a-proti.html
626 / 4536 / 395324-problematicky-gentleman.html
627 / 4536 / 395325-rasputin.html
628 / 4536 / 395326-tu-ten-kamen.html
629 / 4536 / 395327-unos-bankere-fuxe.html
630 / 4536 / 395328-za-oponou-smrti.html
631 / 4536 / 395329-zahadny-pripad-galginuv.html
632 / 4536 / 395330-zavet-podivinova.html
633 / 4536 / 395331-bily-raj.html
634 / 4536 / 395332-chytte-ho.html
635 / 4536 / 395333-dama-z-baru.html
636 / 4536 / 395334-devce-z-hor.html
637 / 4536 / 395335-dvoji-zivot.html
638 / 4536 / 395336-hrichy-v-manzelstvi.html
639 / 4536 / 395337-jindra-hrabenka-ostrovinova.html
640 / 4536 / 395338-kuryri-cis

813 / 4536 / 395512-pradlenka-jeho-jasnosti.html
814 / 4536 / 395513-tajemstvi-lekarovo.html
815 / 4536 / 395514-utrpeni-sede-sestry.html
816 / 4536 / 395515-v-kropenate-lhote-hori.html
817 / 4536 / 395516-vendelinuv-ocistec-a-raj.html
818 / 4536 / 395517-vse-pro-lasku.html
819 / 4536 / 395518-za-rodnou-hroudu.html
820 / 4536 / 395519-rada-lekare.html
821 / 4536 / 395520-tonka-sibenice.html
822 / 4536 / 395521-film-se-sasou-rasilovem-a-karlem-faltysem.html
823 / 4536 / 395522-kdyz-struny-lkaji-nemecka-verze.html
824 / 4536 / 395523-on-a-jeho-sestra-nemecka-verze.html
825 / 4536 / 395524-afera-plukovnika-redla-nemecka-verze.html
826 / 4536 / 395525-c-a-k-polni-marsalek-francouzska-verze.html
827 / 4536 / 395526-zena-ktera-se-smeje.html
828 / 4536 / 395527-svet-bez-hranic.html
829 / 4536 / 395528-afera-plukovnika-redla.html
830 / 4536 / 395529-pana-bumlicka-novy-domov.html
831 / 4536 / 395530-pensionat-petersilia.html
832 / 4536 / 395531-to-neznate-hadimrsku-nemecka-verze.html
833 / 4536

...

3922 / 4536 / 401874-jak-se-dela-film.html
3923 / 4536 / 401875-junaci-v-prirode.html
3924 / 4536 / 401876-kapky-a-bubliny.html
3925 / 4536 / 401877-ladovi-furianti.html
3926 / 4536 / 401878-lide-a-parky.html
3927 / 4536 / 401879-miliony-na-dlazbach.html
3928 / 4536 / 401880-narodni-umelec-josef-lada.html
3929 / 4536 / 401881-nevime-dne.html
3930 / 4536 / 401882-obraz.html
3931 / 4536 / 401883-sklenena-suita.html
3932 / 4536 / 401884-to-jsou-bratri-v-triku.html
3933 / 4536 / 401885-cena-za-stesti.html
3934 / 4536 / 401886-letni-hokej.html
3935 / 4536 / 401887-podkarpatska-rus.html
3936 / 4536 / 401888-posadka-na-stite.html
3937 / 4536 / 401889-pred-startem-do-vesmiru.html
3938 / 4536 / 401890-ptak-ktery-kona-vojenskou-sluzbu.html
3939 / 4536 / 401891-reka-zivota-a-smrti.html
3940 / 4536 / 401892-uhori-maji-nabito.html
3941 / 4536 / 401893-na-strese.html
3942 / 4536 / 401896-skleneny-chleb.html
3943 / 4536 / 401897-strevicek.html
3944 / 4536 / 401898-sumavska-suita.html
3945 / 4536 / 40

4110 / 4536 / 402070-kralove-videa.html
4111 / 4536 / 402071-kiruna-prekrasny-novy-svet.html
4112 / 4536 / 402072-milenec-leta.html
4113 / 4536 / 402073-cesta-do-nemozna.html
4114 / 4536 / 402075-svet-podle-muchy.html
4115 / 4536 / 402077-poslouchej.html
4116 / 4536 / 402078-zenska-pomsta.html
4117 / 4536 / 402079-modelar.html
4118 / 4536 / 402080-mlsne-medvedi-pribehy-v-kinech.html
4119 / 4536 / 402081-v-siti-18.html
4120 / 4536 / 402082-alchymicka-pec.html
4121 / 4536 / 402083-jan-jedlicka-stopy-krajiny.html
4122 / 4536 / 402084-frem.html
4123 / 4536 / 402085-nebe.html
4124 / 4536 / 402086-nova-sichta.html
4125 / 4536 / 402087-zmatek.html
4126 / 4536 / 402088-provincni-mestecko-e.html
4127 / 4536 / 402089-anny.html
4128 / 4536 / 402090-vysehrad-seryjal.html
4129 / 4536 / 402091-princezna-zakleta-v-case.html
4130 / 4536 / 402092-sarlatan.html
4131 / 4536 / 402093-les.html
4132 / 4536 / 402094-babovky.html
4133 / 4536 / 402095-bourak.html
4134 / 4536 / 402096-daria.html
4135 / 4536 / 4

4307 / 4536 / 53055-treti-statocny-jaro-vojtek-rojkoviasnilci.html
4308 / 4536 / 53057-stvrty-statocny-tereza-kucerova-prazske-nokturno-aneb-komora.html
4309 / 4536 / 53059-piaty-statocny-martin-repka-zlodeji.html
4310 / 4536 / 53102-siesty-statocny-aurel-klimt-o-kouzelnem-zvonu.html
4311 / 4536 / 53125-vsichni-moji-blizci.html
4312 / 4536 / 53139-z-pekla-stesti.html
4313 / 4536 / 53143-venkovsky-ucitel.html
4314 / 4536 / 53335-klicek.html
4315 / 4536 / 53571-cernobila-v-barve.html
4316 / 4536 / 53944-kulicky.html
4317 / 4536 / 53961-hannibal-zrozeni.html
4318 / 4536 / 53971-tulaci.html
4319 / 4536 / 54020-zlati-uhori.html
4320 / 4536 / 54060-certe-drz-se-sveho-kopyta.html
4321 / 4536 / 54067-malvina.html
4322 / 4536 / 54076-pan-hradu.html
4323 / 4536 / 54085-vohnice-a-kilijan.html
4324 / 4536 / 54091-dame-kord-neslusi.html
4325 / 4536 / 54224-vaclav.html
4326 / 4536 / 54226-goyovy-prizraky.html
4327 / 4536 / 54236-vy-nam-taky-sefe.html
4328 / 4536 / 54269-nejkrasnejsi-hadanka.html
432

4507 / 4536 / 9149-lotrando-a-zubejda.html
4508 / 4536 / 9151-maskin-zabil-koskina.html
4509 / 4536 / 91578-hodinu-nevis.html
4510 / 4536 / 91594-male-lzi.html
4511 / 4536 / 9168-rychle-pohyby-oci.html
4512 / 4536 / 91819-ocas-jesterky.html
4513 / 4536 / 91952-zaby.html
4514 / 4536 / 9214-ptak-ohnivak.html
4515 / 4536 / 9215-orbis-pictus.html
4516 / 4536 / 9222-modre-z-nebe.html
4517 / 4536 / 9238-suzanne.html
4518 / 4536 / 9241-hranice-stinu.html
4519 / 4536 / 9242-bumerang.html
4520 / 4536 / 9290-jmeno-kodu-rubin.html
4521 / 4536 / 9295-konto-separato.html
4522 / 4536 / 9296-lea.html
4523 / 4536 / 9358-passage.html
4524 / 4536 / 9371-akaty-bile.html
4525 / 4536 / 9462-bajecna-leta-pod-psa.html
4526 / 4536 / 9463-nejasna-zprava-o-konci-sveta.html
4527 / 4536 / 9732-cesta-pustym-lesem.html
4528 / 4536 / 9733-hodiny-na-vezi.html
4529 / 4536 / 9734-mandragora.html
4530 / 4536 / 9735-o-perlove-panne.html
4531 / 4536 / 9736-vychova-divek-v-cechach.html
4532 / 4536 / 9737-hotel-sarajevo-ane

,Film,Země původu,Copyright,Rok výroby,Premiéra,Minutáž,Režie,Kategorie,Žánr,Typologie,...,Stav filmu,Podnázev,Zpěv role,Prodej práv,Podnázev anglický,Výrobní skupina,Zákazník,Vývozní společnost,Rychlost,Přístupnost
0,Tábor padlých žen (10144),"Slovensko, ...",1997,1997,premiéra 28. 9. 2000 /doporučená přístupnost o...,100 min,Laco Halama,film,"historický, tragikomedie","hraný, distribuční",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Eastern – Krvavý Hugo (10173),Česká republika,1997,1997,nedistribuční premiéra 27. 4. 1997 ...,6 min,Aurel Klimt (1189),film,groteska,"animovaný, distribuční, studentský",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,O Ječmínkovi (102282),Česká republika,2003,2002,nedistribuční premiéra 18. 4. 2003 (televizní...,61 min,Milan Cieslar (3593),film,pohádka,"hraný, distribuční",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Zimní víla (102314),Česká republika,1999,1999,nedistribuční premiéra 24. 12. 1999 (televizn...,52 min,Kryštof Hanzlík (1055),film,pohádka,"hraný, distribuční",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Narkoblues (10242),"Česká republika, ...",1997,1997,premiéra 10. 11. 2016 /přístupné bez omezení/,9 min,"Břetislav Pojar (3113), Ivan Vít (3030)",film,osvětový,"animovaný, distribuční",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4531,Hotel Sarajevo aneb Sny obyčejného svatebčana ...,Česká republika,1997,1997,premiéra 12. 8. 1997,NaN,Rodrigo Morales (3466),film,povídka,"hraný, distribuční",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4532,Zdivočelá země (9738),Česká republika,1997,1997,premiéra 13. 11. 1997 /přístupný/ (kino 64 U H...,114 min,Hynek Bočan (2947),film,"dobrodružný, historický","hraný, distribuční",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4533,Biograf (9769),Česká republika,1997,1997,nedistribuční premiéra 12. 6. 1997 ...,5 min,Galina Hůlová (7957),film,anekdota,"animovaný, distribuční, studentský",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4534,Knoflíkáři (9793),Česká republika,1997,1997,premiéra 20. 11. 1997 /doporučená přístupnost ...,102 min,Petr Zelenka (4806),film,"komedie, povídkový","hraný, distribuční",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
df.columns

Index(['Film', 'Země původu', 'Copyright', 'Rok výroby', 'Premiéra', 'Minutáž',
       'Režie', 'Kategorie', 'Žánr', 'Typologie',
       ...
       'Stav filmu', 'Podnázev', 'Zpěv role', 'Prodej práv',
       'Podnázev anglický', 'Výrobní skupina', 'Zákazník',
       'Vývozní společnost', 'Rychlost', 'Přístupnost'],
      dtype='object', length=105)

## Export

In [24]:
df.to_json("filmy_raw.json", orient="records")